In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [5]:
def gather_outputs(outputs: list) -> list:
    # Group entities by their sequence
    grouped_entities = []
    current_group = []
    for entity in outputs:
        if not current_group or entity['start'] == current_group[-1]['end']:
            current_group.append(entity)
        else:
            grouped_entities.append(current_group)
            current_group = [entity]

    # Append the last group
    if current_group:
        grouped_entities.append(current_group)

    return grouped_entities

def get_locations_from_outputs(sentence: str, outputs: list) -> list:
    groups = gather_outputs(outputs)
    return [sentence[group[0]["start"]:group[-1]["end"]] for group in groups]

In [6]:
from transformers import pipeline

sentence = "Je veux aller de Port-Boulet à Le Havre."
#sentence = "Peux-tu m'aider à trouver mon chemin de Paris à Épierre ?"

token_classifier = pipeline("token-classification", model="models/distilbert-finetuned-token-classification-ner-trip", aggregation_strategy="simple")
outputs = token_classifier(sentence)

In [7]:
get_locations_from_outputs(sentence, outputs)

['Port-Boulet', 'Le Havre']